In [35]:

import sys
sys.path.append('../../src/scenes/')
from tools import _get_bsc, _get_gaia
import numpy as np

import pandas as pd 
from tools import _get_tyco, _get_gaia, query_ps1
from astropy.coordinates import SkyCoord, Angle

tess = pd.read_csv('../TESS_FFI/TESS_FFI_Coordinates.csv')


In [60]:
tess[tess['Sector']==20]

,Sector,Camera,CCD,RA_center,DEC_center,ROTATION,RA_corner1,DEC_corner1,RA_corner2,DEC_corner2,RA_corner3,DEC_corner3,RA_corner4,DEC_corner4
304,20,1,1,120.178186,45.119140,164.068454,130.674939,48.531341,125.591335,37.651859,110.239488,40.447003,113.010255,52.200010
305,20,1,2,102.262503,47.063336,177.842172,112.732027,52.221646,110.014280,40.461638,93.869788,41.062852,93.324023,52.542626
306,20,1,3,101.452726,34.781820,-3.525029,94.616351,29.345373,93.878267,40.817011,109.961236,40.206813,108.070511,28.398079
307,20,1,4,116.470629,33.145847,-11.142112,108.267637,28.375109,110.184607,40.179423,125.487327,37.406595,121.406443,26.447778
308,20,2,1,137.513963,67.072379,146.126104,158.106857,67.367912,141.119148,58.739172,117.879587,63.991899,129.721063,75.216734
309,20,2,2,103.622426,70.925458,178.786693,129.125780,75.300701,117.485120,64.052265,89.157421,64.389157,80.868242,75.544523
310,20,2,3,102.854697,58.611270,-2.025480,93.186120,52.817470,89.287821,64.122507,117.330963,63.773005,112.601579,52.133607
311,20,2,4,125.731886,56.059926,-20.512057,112.867998,52.100783,117.720903,63.730117,140.819581,58.520879,130.673350,48.707846
312,20,3,1,104.227184,82.507131,0.364580,80.517592,75.778918,28.305391,83.765230,181.310013,83.665500,129.126923,75.286433
313,20,3,2,163.684526,75.403916,-57.454944,129.688864,75.215312,181.611746,83.506245,190.315169,71.395558,158.879815,67.593791


In [ ]:
import os
from glob import glob
def _Save_space(Save,delete=False):
    """
    Creates a path if it doesn't already exist.
    """
    try:
        os.makedirs(Save)
    except FileExistsError:
        if delete:
            os.system(f'rm -r {Save}/')
            os.makedirs(Save)
        else:
            pass

In [57]:
def bulk_download_catalogs(fields,savepath,overwrite=False,sector=None):
    tess = pd.read_csv(fields)
    if sector is not None:
        tess = tess.loc[tess.Sector == sector]
    center = SkyCoord(tess.RA_center.values,tess.DEC_center.values,unit='deg')
    dists = np.zeros((len(center),4))
    for i in range(3):
        i += 1
        corn = SkyCoord(tess[f'RA_corner{i}'].values,tess[f'DEC_corner{i}'].values,unit='deg')
        dists[:,i-1] = center.separation(corn).deg
    rads = np.nanmax(dists,axis=1) + 0.4
    

    for i in range(len(rads)):
        sp = savepath + f'Sector{tess.Sector.iloc[i]}/'
        _Save_space(sp)
        name = f'Sector{tess.Sector.iloc[i]}_ccd{4*(tess.Camera.iloc[i]-1) + tess.CCD.iloc[i]}'
        #if check_exist(savepath+savepath+name+'_ps1.csv',overwrite):
        #    if center[i].ra.deg > -30:
        #        ps1 = query_ps1(center[i].ra.deg,center[i].dec.deg,rads[i])
        #        ps1.to_csv(savepath+name+'_ps1.csv')
        if (len(glob(sp+name+'_gaia.csv')) == 0) | overwrite:
            gaia = _get_gaia(center[i],rads[i])
            gaia.to_csv(sp+name+'_gaia.csv',index=False)
        if (len(glob(sp+name+'_bsc.csv')) == 0) | overwrite:
            bsc = _get_bsc(center[i],rads[i])
            bsc.to_csv(sp+name+'_bsc.csv',index=False)


In [58]:
savepath = 'catalogs/'
bulk_download_catalogs('../TESS_FFI/TESS_FFI_Coordinates.csv',savepath,sector=20)

KeyboardInterrupt: 

In [65]:
import mastcasjobs
import os

def ps1_casjobs(coord,radius,maglim=20):
    query = f"""
            SELECT o.objID,
            o.raMean, o.decMean,
            o.qualityFlag,
            o.gMeanPSFMag, o.gMeanPSFMagErr, o.gMeanPSFMagNpt,
            o.rMeanPSFMag, o.rMeanPSFMagErr, o.rMeanPSFMagNpt,
            o.iMeanPSFMag, o.iMeanPSFMagErr, o.iMeanPSFMagNpt,
            o.zMeanPSFMag, o.zMeanPSFMagErr, o.zMeanPSFMagNpt,
            o.yMeanPSFMag, o.yMeanPSFMagErr, o.yMeanPSFMagNpt,
            o.rMeanKronMag, o.rMeanKronMagErr,
            o.iMeanKronMag, o.iMeanKronMagErr,
            o.nDetections,
            o.gFlags, o.gQfPerfect,
            o.rFlags, o.rQfPerfect,
            o.iFlags, o.iQfPerfect,
            o.zFlags, o.zQfPerfect,
            o.yFlags, o.yQfPerfect
            INTO mydb.[syndiff_test_field2]
            from fGetNearbyObjEq({coord.ra.deg}, {coord.dec.deg}, {radius}*60) as x
            JOIN MeanObjectView o on o.ObjID=x.ObjId
            LEFT JOIN StackObjectAttributes AS soa ON soa.objID = x.objID
            WHERE o.nDetections>5
            AND soa.primaryDetection>0
            AND o.iMeanPSFMag < 20
            """
    try:
        jobs = mastcasjobs.MastCasJobs(context="PanSTARRS_DR2")
        results = jobs.submit(query, task_name="SynDiff field cat",estimate=70)
        results = results.to_pandas()
    except:
        results = None
    return results

In [69]:
rads[304]

8.686020185961805

In [66]:
cat = ps1_casjobs(center[304],rads[304])

Exception: ExecuteQuickJob failed with status: 500
Query exceeds queue time.  Please revise your query or use a longer queue. 

In [67]:
jobs = mastcasjobs.MastCasJobs(context="PanSTARRS_DR2")

In [ ]:
jobs.submit()

In [26]:


center = SkyCoord(tess.RA_center.values,tess.DEC_center.values,unit='deg')
dists = np.zeros((len(center),4))
for i in range(3):
    i += 1
    corn = SkyCoord(tess[f'RA_corner{i}'].values,tess[f'DEC_corner{i}'].values,unit='deg')
    dists[:,i-1] = center.separation(corn).deg
rads = np.nanmax(dists,axis=1) + 2/60

In [38]:
tess[tess['Sector'] == 20]

,Sector,Camera,CCD,RA_center,DEC_center,ROTATION,RA_corner1,DEC_corner1,RA_corner2,DEC_corner2,RA_corner3,DEC_corner3,RA_corner4,DEC_corner4
304,20,1,1,120.178186,45.119140,164.068454,130.674939,48.531341,125.591335,37.651859,110.239488,40.447003,113.010255,52.200010
305,20,1,2,102.262503,47.063336,177.842172,112.732027,52.221646,110.014280,40.461638,93.869788,41.062852,93.324023,52.542626
306,20,1,3,101.452726,34.781820,-3.525029,94.616351,29.345373,93.878267,40.817011,109.961236,40.206813,108.070511,28.398079
307,20,1,4,116.470629,33.145847,-11.142112,108.267637,28.375109,110.184607,40.179423,125.487327,37.406595,121.406443,26.447778
308,20,2,1,137.513963,67.072379,146.126104,158.106857,67.367912,141.119148,58.739172,117.879587,63.991899,129.721063,75.216734
309,20,2,2,103.622426,70.925458,178.786693,129.125780,75.300701,117.485120,64.052265,89.157421,64.389157,80.868242,75.544523
310,20,2,3,102.854697,58.611270,-2.025480,93.186120,52.817470,89.287821,64.122507,117.330963,63.773005,112.601579,52.133607
311,20,2,4,125.731886,56.059926,-20.512057,112.867998,52.100783,117.720903,63.730117,140.819581,58.520879,130.673350,48.707846
312,20,3,1,104.227184,82.507131,0.364580,80.517592,75.778918,28.305391,83.765230,181.310013,83.665500,129.126923,75.286433
313,20,3,2,163.684526,75.403916,-57.454944,129.688864,75.215312,181.611746,83.506245,190.315169,71.395558,158.879815,67.593791


In [37]:
center

<SkyCoord (ICRS): (ra, dec) in deg
    [(319.4839875 , -41.03857318), (334.47465871, -36.43184132),
     (328.6997054 , -25.17258034), ..., (348.59449737,  77.57044661),
     ( 32.00363158,  71.94393399), ( 76.43007003,  76.93618976)]>

In [50]:
savepath = 'catalogs/'
i = 304
savepath += f'Sector{tess.Sector.iloc[i]}/'
_Save_space(savepath)
name = f'Sector{tess.Sector.iloc[i]}_ccd{4*(tess.Camera.iloc[i]-1) + tess.CCD.iloc[i]}'
#if check_exist(savepath+savepath+name+'_ps1.csv',overwrite):
#    if center[i].ra.deg > -30:
#        ps1 = query_ps1(center[i].ra.deg,center[i].dec.deg,rads[i])
#        ps1.to_csv(savepath+name+'_ps1.csv')
if len(glob(savepath+name+'_gaia.csv')) == 0:
    gaia = _get_gaia(center[i],rads[i])
    gaia.to_csv(savepath+name+'_gaia.csv',index=False)
if len(glob(savepath+name+'_bsc.csv')) == 0:
    bsc = _get_bsc(center[i],rads[i])
    bsc.to_csv(savepath+name+'_bsc.csv',index=False)

In [48]:
savepath

'catalogs/Sector20/Sector20/Sector20/'

In [30]:
tess

,Sector,Camera,CCD,RA_center,DEC_center,ROTATION,RA_corner1,DEC_corner1,RA_corner2,DEC_corner2,RA_corner3,DEC_corner3,RA_corner4,DEC_corner4
0,1,1,1,319.483987,-41.038573,16.178684,313.777217,-47.871234,310.092203,-36.701889,324.468061,-33.330235,330.129431,-44.413344
1,1,1,2,334.474659,-36.431841,26.415267,330.344064,-44.336617,324.664608,-33.277880,337.720584,-28.385630,344.128531,-38.562134
2,1,1,3,328.699705,-25.172580,-158.671958,332.049749,-17.867487,337.598475,-28.164441,324.550516,-33.034760,320.192557,-21.767718
3,1,1,4,315.326394,-29.203484,-163.994506,320.013395,-21.820102,324.359209,-33.090412,310.044938,-36.449224,307.641963,-25.139192
4,1,2,1,333.207418,-63.423190,24.364041,324.750778,-70.676065,317.244403,-59.600483,338.371126,-54.343546,352.506681,-64.922769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,79,3,4,253.706124,79.158868,-179.894578,305.020796,82.168925,274.195324,72.047745,233.675261,72.144535,206.505038,82.224912
1260,79,4,1,85.154904,89.247550,153.568387,205.132069,82.457130,118.233155,80.889959,33.683317,80.901775,304.940766,82.082144
1261,79,4,2,348.594497,77.570447,-92.047279,305.649440,81.947453,32.869645,80.799265,5.308080,70.358777,329.929774,71.303045
1262,79,4,3,32.003632,71.943934,42.612310,31.282841,64.016190,6.098719,70.257742,34.102021,80.576122,57.686599,70.057228


array([8.66700928, 8.64062629, 8.6553799 , ..., 8.64507479, 8.64516928,
       8.64140968])

In [12]:
i = 1
corn = SkyCoord(tess[f'RA_corner{i}'].values,tess[f'DEC_corner{i}'].values,unit='deg')
corners += [corn]
rads = center.separation(corn).deg + 2/60

In [15]:
len(corn)

1264

In [16]:
dist.shape

(1264,)

In [6]:
center

<SkyCoord (ICRS): (ra, dec) in deg
    [(319.4839875 , -41.03857318), (334.47465871, -36.43184132),
     (328.6997054 , -25.17258034), ..., (348.59449737,  77.57044661),
     ( 32.00363158,  71.94393399), ( 76.43007003,  76.93618976)]>

In [ ]:



sc = SkyCoord(self.ps1.ra,self.ps1.dec, frame='icrs', unit='deg')
gaia = 
bsc = _get_bsc(sc,0.41)